# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [6]:
from decimal import Decimal, getcontext
from contextlib import contextmanager
from typing import Generator
@contextmanager
def set_precision(precision : int) -> Generator[None, None, None]:
    precision = int(precision)

    if (precision < 0):
        raise ValueError("Precision should be non negative")
    
    #сохранение контекста

    precision_old = getcontext().prec

    getcontext().prec = precision

    #возвращение управления

    try:
        yield
    
    #восстановление контекста
    finally:
        getcontext().prec = precision_old
    

In [7]:
num1, num2 = Decimal("3.14"), Decimal("2.72")
print(num1 / num2)

with set_precision(19):
    print(num1/num2)

1.154411764705882352941176471
1.154411764705882353


## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [8]:
from typing import Generator


def generate_arithmetic_progression(start=1, step=1) -> Generator:
    current_member, step = float(start), float(step)

    while True:
        yield current_member
        current_member *= step

In [9]:
progression_gen = generate_arithmetic_progression(step=0.5)
for i in range(5):
    print(f'member_{i + 1}: {next(progression_gen)};')

member_1: 1.0;
member_2: 0.5;
member_3: 0.25;
member_4: 0.125;
member_5: 0.0625;


## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [3]:
def float_range(stop, start = 0, step = 1):
    if step == 0:
        raise ValueError("range arg3 cloud non zero")
    current_member = start
    while True:
        if ((current_member >= stop) and step > 0) or ((stop >= current_member) and step < 0):
            return
        yield current_member            
        current_member += step

In [4]:
for i in float_range(start = 1, stop = 0, step = -0.5):
    print(i)
for i in float_range(stop=1,step=0.5):
    print(i)

1
0.5
0
0.5


## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [24]:
def my_map(func, *iterables):
    for item in zip(*iterables):
        yield func(*item)


[3, 6, 9]


### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [34]:
from enum import Enum
from itertools import zip_longest

class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


def new_my_zip(func,*iterables,  map_types=MapTypes.LONGEST):
    if map_types.value == "short":
        for item in zip(*iterables):
            yield func(*item)
    elif map_types.value == "long":
        for item in zip_longest(*iterables):
            yield func(*item)
    else: 
        raise ValueError("некоректное значение map_types")


## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [2]:
from typing import Generator
def generate_circle(spiner_string = "abc") -> Generator :
    if spiner_string != "":
        current_step = -1
        while True:
            current_step += 1
            if current_step == len(spiner_string):
                current_step = 0
            yield spiner_string[current_step]
    else:
        raise ValueError("string cloud non '' ")


### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [9]:
import time



def delat_koleso_Basta(time_limit=4, pause=1):
    '''
    Баста, скриптонит - Сансара
    Когда меня не станет — я буду петь голосами
    Моих детей и голосами их детей
    Нас просто меняют местами
    Таков закон сансары, круговорот людей
    О-о-ой, мама...
    Когда меня не станет — я буду петь голосами
    Моих детей и голосами их детей
    Нас просто меняют местами
    Таков закон сансары, круговорот людей
    О-о-ой, мама...
    '''
    spiner_string = "|/-\\"
    gen = generate_circle(spiner_string)
    timeout = time.time() + time_limit
    amount_itteration = time_limit/pause
    message = "Thinking: "

    print(message, next(gen), end="\r")
    
    if amount_itteration <= 1:
        while True:
            if time.time() >= timeout:
                break
    else:
        for _ in range(int(amount_itteration)):
            time.sleep(pause)
            print(message, next(gen), end="\r")
        while True:
            if time.time() >= timeout:
                break
